<a href="https://colab.research.google.com/github/czarinadamienne/MAGNET/blob/main/MAGNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### C Code
**Artificial Test Dataset Used -**https://docs.google.com/spreadsheets/d/1KaqRN2SzVxtRMPvIvwpXUomcN41FDrAqoWW_88X1owM/edit?usp=sharing

**Manual Computation Testing -**
https://docs.google.com/spreadsheets/d/1P4VrBK_uCHIrjgCkf5wXhvakFBDHCyEU/edit?usp=sharing&ouid=108923980891918456197&rtpof=true&sd=true

In [15]:
%%writefile magnet.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>
#include <time.h>

#define MAX_SEQ_LEN 1024
#define MAX_SEQUENCES 1000
#define STACK_SIZE 1000

typedef struct
{
    int* lengths;
    int* starts;
    int* ends;
    int count;
} ConsecutiveZerosInfo;

typedef struct{
  int start;
  int end;
} Range;

ConsecutiveZerosInfo ConsecutiveZeros(int* sequence, int length);
void Extraction_Encapsulation(int** HammingMask, int start, int end, int* MagnetMask, int ErrorThreshold, int read_len, int* accepted);
int MAGNET(const char* RefSeq, const char* ReadSeq, int ErrorThreshold, int* finalBitVector, int** HammingMask, int* accepted);
int readCSVFile(char sequences[MAX_SEQUENCES][MAX_SEQ_LEN], char refSequences[MAX_SEQUENCES][MAX_SEQ_LEN], int editThresholds[MAX_SEQUENCES], int *numSequences);
void printStats(int acceptedCount, int rejectedCount, int errors, int numSequences, double elapse);

void printStats(int acceptedCount, int rejectedCount, int errors, int numSequences, double elapse)
{
  printf("----- Statistics -----\n\n");
  printf("Number of accepted sequences: %d\n", acceptedCount);
  printf("Number of rejected sequences: %d\n", rejectedCount);
  printf("Number of sequences processed: %d\n", numSequences);
  printf("Number of runs: 10\n");
  printf("Average time taken: %f seconds\n", elapse / 10);
  printf("Number of errors: %d\n", errors);
}

ConsecutiveZerosInfo ConsecutiveZeros(int* sequence, int length)
{
    int* starts = (int*)malloc(length * sizeof(int));
    int* ends = (int*)malloc(length * sizeof(int));
    int* lengths = (int*)malloc(length * sizeof(int));

    int count = 0;
    int in_zero_run = 0;
    int start_idx = 0;

    for (int i = 0; i < length; i++)
    {
        if (sequence[i] == 0)
        {
            if (!in_zero_run)
            {
                start_idx = i;
                in_zero_run = 1;
            }
        }
        else
        {
            if (in_zero_run)
            {
                ends[count] = i - 1;
                starts[count] = start_idx;
                lengths[count] = ends[count] - starts[count] + 1;
                count++;
                in_zero_run = 0;
            }
        }
    }

    if (in_zero_run)
    {
        ends[count] = length - 1;
        starts[count] = start_idx;
        lengths[count] = ends[count] - starts[count] + 1;
        count++;
    }

    return (ConsecutiveZerosInfo){lengths, starts, ends, count};
}

void Extraction_Encapsulation(int** HammingMask, int start, int end, int* MagnetMask, int ErrorThreshold, int read_len, int* accepted) {
    Range stack[STACK_SIZE];
    int top = -1;

    stack[++top] = (Range){start, end};

    while (top >= 0) {
        Range current = stack[top--];
        start = current.start;
        end = current.end;

        if (start > end) continue;

        int max_run = 0, max_start = -1, max_end = -1;
        int second_max_start = -1, second_max_end = -1;

        for (int mask_id = 0; mask_id <= 2 * ErrorThreshold; mask_id++) {
            int sub_len = end - start + 1;
            int* sub_seq = (int*)malloc(sub_len * sizeof(int));

            for (int i = 0; i < sub_len; i++) {
                sub_seq[i] = HammingMask[mask_id][start + i];
            }

            ConsecutiveZerosInfo info = ConsecutiveZeros(sub_seq, sub_len);
            free(sub_seq);

            for (int i = 0; i < info.count; i++) {
                if (info.lengths[i] > max_run) {
                    second_max_start = max_start;
                    second_max_end = max_end;

                    max_run = info.lengths[i];
                    max_start = start + info.starts[i];
                    max_end = start + info.ends[i];
                }
            }

            free(info.lengths);
            free(info.starts);
            free(info.ends);
        }

        if (max_run == 0 || max_start == -1 || max_end == -1) continue;

        for (int i = max_start; i <= max_end; i++) {
            MagnetMask[i] = 0;
        }

        if (second_max_start != -1 && second_max_end != -1) {
            if (second_max_start > max_end + 1) {
                MagnetMask[max_end + 1] = 1;
                MagnetMask[max_end + 2] = 1;
            }
        } else if (max_end + 1 < read_len) {
            MagnetMask[max_end + 1] = 1;
        }

        if (max_start - 2 >= start) {
            stack[++top] = (Range){start, max_start - 2};
        }

        if (max_end + 2 <= end) {
            stack[++top] = (Range){max_end + 2, end};
        }
    }
}

int MAGNET(const char* RefSeq, const char* ReadSeq, int ErrorThreshold, int* finalBitVector, int** HammingMask, int* accepted)
{
    int read_len = strlen(ReadSeq);
    int max_masks = 2 * ErrorThreshold + 1;
    *accepted = 0;

    // Initialize Hamming masks
    for (int i = 0; i < max_masks; i++)
    {
        HammingMask[i] = (int*)calloc(read_len, sizeof(int));
    }

    // Initialize MagnetMask to 1s
    int* MagnetMask = (int*)malloc(read_len * sizeof(int));
    for (int i = 0; i < read_len; i++) MagnetMask[i] = 1;

    int error_count = 0;
    for (int i = 0; i < read_len; i++)
    {
        HammingMask[0][i] = (ReadSeq[i] != RefSeq[i]);
        error_count += HammingMask[0][i];
    }

    if (error_count <= ErrorThreshold)
    {
        *accepted = 1;
        memcpy(finalBitVector, MagnetMask, read_len * sizeof(int));
        free(MagnetMask);
        return *accepted;
    }

    //Right-shifted masks (deletions)
    for (int e = 1; e <= ErrorThreshold; e++)
    {
        error_count = 0;
        memset(HammingMask[e], 0, read_len * sizeof(int));
        for (int i = e; i < read_len; i++)
        {
            if (i - e < strlen(ReadSeq))
            {
                HammingMask[e][i] = (ReadSeq[i - e] != RefSeq[i]);
                error_count += HammingMask[e][i];
            }
        }

        if (error_count <= ErrorThreshold)
        {
            *accepted = 1;
            memcpy(finalBitVector, MagnetMask, read_len * sizeof(int));
            free(MagnetMask);
            return *accepted;
        }
    }

    //Left-shifted masks (insertions)
    for (int e = 1; e <= ErrorThreshold; e++)
    {
        error_count = 0;
        memset(HammingMask[ErrorThreshold + e], 0, read_len * sizeof(int));
        for (int i = 0; i < read_len - e; i++)
        {
            if (i + e < strlen(ReadSeq))
            {
                HammingMask[ErrorThreshold + e][i] = (ReadSeq[i + e] != RefSeq[i]);
                error_count += HammingMask[ErrorThreshold + e][i];
            }
        }
        if (error_count <= ErrorThreshold)
        {
            *accepted = 1;
            memcpy(finalBitVector, MagnetMask, read_len * sizeof(int));
            free(MagnetMask);
            return *accepted;
        }
    }


    Extraction_Encapsulation(HammingMask, 0, read_len - 1, MagnetMask, ErrorThreshold, read_len, accepted);

    memcpy(finalBitVector, MagnetMask, read_len * sizeof(int));

    error_count = 0;
    for (int i = 0; i < read_len; i++)
    {
        error_count += MagnetMask[i];
    }

    *accepted = (error_count <= ErrorThreshold) ? 1 : 0;
    free(MagnetMask);
    return *accepted;
}

int readCSVFile(char sequences[MAX_SEQUENCES][MAX_SEQ_LEN], char refSequences[MAX_SEQUENCES][MAX_SEQ_LEN], int editThresholds[MAX_SEQUENCES], int *numSequences)
{
    FILE *file = fopen("dataset.csv", "r");
    if (!file)
    {
        printf("no file\n");
        return 0;
    }

    char line[MAX_SEQ_LEN];
    *numSequences = 0;
    while (fgets(line, sizeof(line), file) && *numSequences < MAX_SEQUENCES)
    {
        sscanf(line, "%[^,],%[^,],%d", sequences[*numSequences], refSequences[*numSequences], &editThresholds[*numSequences]);
        (*numSequences)++;
    }
    fclose(file);
    return 1;
}

int main()
{
    char (*sequences)[MAX_SEQ_LEN] = malloc(MAX_SEQUENCES * MAX_SEQ_LEN * sizeof(char));
    char (*refSequences)[MAX_SEQ_LEN] = malloc(MAX_SEQUENCES * MAX_SEQ_LEN * sizeof(char));
    int* editThresholds = malloc(MAX_SEQUENCES * sizeof(int));
    int numSequences;
    int accepted = 0;
    int acceptedC = 0;
    int errors = 0;
    int acceptedCount = 0;
    int rejectedCount = 0;
    int run = 10;
    double time = 0.0;
    double elapse = 0.0;
    int finalBitVector[MAX_SEQ_LEN];

    if (!readCSVFile(sequences, refSequences, editThresholds, &numSequences))
    {
        return 1;
    }

    for (int i = 0; i < numSequences; i++){
      int maskSize = 2 * editThresholds[i] + 1;
      int** HammingMask = (int**)malloc(maskSize * sizeof(int*));

      int cachemiss = MAGNET(refSequences[i], sequences[i], editThresholds[i], finalBitVector, HammingMask, &accepted);
    }

    //int* accepted = 0;

    for (int j = 0; j < run; j++) {
        acceptedCount = 0;
        rejectedCount = 0;
        clock_t start, end;

        for (int i = 0; i < numSequences; i++) {
            int** HammingMask = (int**)malloc((2 * editThresholds[i] + 1) * sizeof(int*));

            for (int e = 0; e < 2 * editThresholds[i] + 1; e++)
            {
                HammingMask[e] = (int*)malloc(MAX_SEQ_LEN * sizeof(int));
                memset(HammingMask[e], 0, MAX_SEQ_LEN * sizeof(int));
            }

            //reset
            memset(finalBitVector, 0, sizeof(finalBitVector));
            int finalBitVectorC[MAX_SEQ_LEN];
            memset(finalBitVectorC, 0, sizeof(finalBitVectorC));

            start = clock();

            int result = MAGNET(refSequences[i], sequences[i], editThresholds[i], finalBitVector, HammingMask, &accepted);
            int checker = MAGNET(refSequences[i], sequences[i], editThresholds[i], finalBitVectorC, HammingMask, &acceptedC);

            end = clock();
            time = (double)(end - start) / CLOCKS_PER_SEC;
            elapse += time;

            if (result) {
                acceptedCount++;
            } else {
                rejectedCount++;
            }

            for (int k = 0; k < strlen(sequences[i]); k++) {
                if (finalBitVector[k] != finalBitVectorC[k]) {
                    errors++;
                }
            }

            // Free HammingMask memory
            for (int e = 0; e < 2 * editThresholds[i] + 1; e++) {
                free(HammingMask[e]);
            }
            free(HammingMask);
        }

        printf("Run %d Time: %.6f seconds | Accepted: %d | Rejected: %d\n",
               j + 1, time, acceptedCount, rejectedCount);
    }

    printf("\n");
    printStats(acceptedCount, rejectedCount, errors, numSequences, elapse);

    free(sequences);
    free(refSequences);
    free(editThresholds);

    return 0;
}

Overwriting magnet.c


In [16]:
!gcc magnet.c -o magnet && ./magnet


Run 1 Time: 0.000045 seconds | Accepted: 346 | Rejected: 654
Run 2 Time: 0.000055 seconds | Accepted: 346 | Rejected: 654
Run 3 Time: 0.000039 seconds | Accepted: 346 | Rejected: 654
Run 4 Time: 0.000041 seconds | Accepted: 346 | Rejected: 654
Run 5 Time: 0.000039 seconds | Accepted: 346 | Rejected: 654
Run 6 Time: 0.000060 seconds | Accepted: 346 | Rejected: 654
Run 7 Time: 0.000039 seconds | Accepted: 346 | Rejected: 654
Run 8 Time: 0.000038 seconds | Accepted: 346 | Rejected: 654
Run 9 Time: 0.000039 seconds | Accepted: 346 | Rejected: 654
Run 10 Time: 0.000039 seconds | Accepted: 346 | Rejected: 654

----- Statistics -----

Number of accepted sequences: 346
Number of rejected sequences: 654
Number of sequences processed: 1000
Number of runs: 10
Average time taken: 0.094689 seconds
Number of errors: 0


### CUDA

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Mar 31 08:06:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
%%writefile magnetCUDA.cu

// biggest changes ig
// [x] uses cudaMallloc to allocate memory on the gpu
// [x] cudaMemcpy is used to transfer data between cpu and gpu
// [x] cudaMallocHost allocates pinned memory that can be transferred more efficiently
// [x] cudaFree and cudaFreeHost for memory cleanup
// [x] cudaDeviceScheduleYield lets the gpu yield to other processes which technically optimizes the performance

// [x] uses grid configuration (dim3) for parallel processing
// [x] uses shared memory and that is the sharedMemSize
// [x] syncthreads() bc it divides to diff threads then it will have to reach the same point before proceeding
// [x] atomicAdd is used for thread-safe updates to counters in global memory

// basically what happens for each sequence:
// transfers data to gpu -> launches kernels to compute alignment -> retrieves results from gpu -> extracts magnet mask -> determines if sequences match

// to test:
// [] super laaaaaarge datasets but still shortreads

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <cuda_runtime.h>

#define MAX_SEQ_LEN 1024
#define MAX_SEQUENCES 1000
#define BLOCK_SIZE 256
#define MAX_ERROR_THRESHOLD 20
#define WARP_SIZE 32

typedef struct {
    int start, end;
} Range;

typedef struct {
    int* lengths;
    int* starts;
    int* ends;
    int count;
} ConsecutiveZerosInfo;

__global__ void computeHammingAndErrorsKernel(const char* refSeq, const char* readSeq,
                                            int* allHammingMasks, int* errorCounts,
                                            int readLen, int errorThreshold) {
    extern __shared__ char sharedMem[];
    char* s_refSeq = sharedMem;
    char* s_readSeq = &sharedMem[readLen];
    int* s_errors = (int*)&sharedMem[2 * readLen];

    int tid = threadIdx.x;
    int maskId = blockIdx.y;
    int idx = blockIdx.x * blockDim.x + tid;

    if (tid < readLen) {
        s_refSeq[tid] = refSeq[tid];
        s_readSeq[tid] = readSeq[tid];
    }

    s_errors[tid] = 0;

    __syncthreads();

    if (idx < readLen && maskId <= 2 * errorThreshold) {
        int shift = 0;
        bool isInsertion = false;

        if (maskId > 0) {
            shift = (maskId + 1) / 2;
            isInsertion = (maskId % 2 == 0);
        }

        int value = 0;

        if (!isInsertion) {
            int srcIdx = idx - shift;
            value = (srcIdx >= 0 && srcIdx < readLen) ? (s_readSeq[srcIdx] != s_refSeq[idx]) : 0;
        } else {
            int srcIdx = idx + shift;
            value = (srcIdx < readLen) ? (s_readSeq[srcIdx] != s_refSeq[idx]) : 0;
        }

        allHammingMasks[maskId * readLen + idx] = value;
        s_errors[tid] = value;
    }

    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            s_errors[tid] += s_errors[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0 && maskId <= 2 * errorThreshold) {
        atomicAdd(&errorCounts[maskId], s_errors[0]);
    }
}

__global__ void findConsecutiveZerosKernel(int* sequence, int length, int* starts, int* ends, int* count) {
    extern __shared__ int s_data[];
    int* s_seq = s_data;
    int* s_transitions = &s_data[blockDim.x];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + tid;

    if (idx < length) {
        s_seq[tid] = sequence[idx];
    } else {
        s_seq[tid] = 1;
    }

    __syncthreads();

    if (tid < blockDim.x - 1 && idx < length - 1) {
        s_transitions[tid] = (s_seq[tid] == 0 && s_seq[tid + 1] != 0) ||
                             (s_seq[tid] != 0 && s_seq[tid + 1] == 0);
    } else {
        s_transitions[tid] = 0;
    }

    if (tid == 0 && idx > 0) {
        s_transitions[0] |= (sequence[idx - 1] == 0 && s_seq[0] != 0) ||
                            (sequence[idx - 1] != 0 && s_seq[0] == 0);
    }

    if (tid == blockDim.x - 1 && idx < length - 1) {
        s_transitions[tid] |= (s_seq[tid] == 0 && sequence[idx + 1] != 0) ||
                              (s_seq[tid] != 0 && sequence[idx + 1] == 0);
    }

    __syncthreads();

    if (s_transitions[tid] && idx < length) {
        if (s_seq[tid] == 0) {
            int pos = atomicAdd(count, 1);
            if (pos < length) {
                ends[pos] = idx;
            }
        } else if (tid < blockDim.x - 1 && s_seq[tid + 1] == 0) {
            int pos = atomicAdd(count, 0);
            if (pos < length && pos > 0) {
                starts[pos - 1] = idx + 1;
            }
        }
    }

    if (tid == 0 && idx == 0 && s_seq[0] == 0) {
        int pos = atomicAdd(count, 1);
        starts[pos - 1] = 0;
    }
}

ConsecutiveZerosInfo findConsecutiveZeros(int* sequence, int length) {
    int* starts = (int*)malloc(length * sizeof(int));
    int* ends = (int*)malloc(length * sizeof(int));
    int* lengths = (int*)malloc(length * sizeof(int));
    int count = 0;
    int inZeroRun = 0;
    int startIdx = 0;

    for (int i = 0; i < length; i++) {
        if (sequence[i] == 0) {
            if (!inZeroRun) {
                startIdx = i;
                inZeroRun = 1;
            }
        } else if (inZeroRun) {
            ends[count] = i - 1;
            starts[count] = startIdx;
            lengths[count] = ends[count] - starts[count] + 1;
            count++;
            inZeroRun = 0;
        }
    }

    if (inZeroRun) {
        ends[count] = length - 1;
        starts[count] = startIdx;
        lengths[count] = ends[count] - starts[count] + 1;
        count++;
    }

    return (ConsecutiveZerosInfo){lengths, starts, ends, count};
}

void extractMagnetMask(int** hammingMasks, int start, int end, int* magnetMask,
                      int errorThreshold, int readLen) {
    Range stack[MAX_SEQ_LEN];
    int top = -1;
    stack[++top] = (Range){start, end};

    while (top >= 0) {
        Range current = stack[top--];
        start = current.start;
        end = current.end;

        if (start > end) continue;

        int maxRunLength = 0, maxStart = -1, maxEnd = -1;
        int secondMaxStart = -1, secondMaxEnd = -1;

        for (int maskId = 0; maskId <= 2 * errorThreshold; maskId++) {
            int subLen = end - start + 1;
            int* subSeq = (int*)malloc(subLen * sizeof(int));
            memcpy(subSeq, &hammingMasks[maskId][start], subLen * sizeof(int));

            ConsecutiveZerosInfo info = findConsecutiveZeros(subSeq, subLen);
            free(subSeq);

            for (int i = 0; i < info.count; i++) {
                if (info.lengths[i] > maxRunLength) {
                    secondMaxStart = maxStart;
                    secondMaxEnd = maxEnd;
                    maxRunLength = info.lengths[i];
                    maxStart = start + info.starts[i];
                    maxEnd = start + info.ends[i];
                }
            }

            free(info.lengths);
            free(info.starts);
            free(info.ends);
        }

        if (maxRunLength == 0 || maxStart == -1) continue;

        for (int i = maxStart; i <= maxEnd; i++) {
            magnetMask[i] = 0;
        }

        if (secondMaxStart != -1 && secondMaxEnd != -1 && secondMaxStart > maxEnd + 1) {
            magnetMask[maxEnd + 1] = magnetMask[maxEnd + 2] = 1;
        } else if (maxEnd + 1 < readLen) {
            magnetMask[maxEnd + 1] = 1;
        }

        if (maxStart - 2 >= start) stack[++top] = (Range){start, maxStart - 2};
        if (maxEnd + 2 <= end) stack[++top] = (Range){maxEnd + 2, end};
    }
}

int magnetCuda(const char* refSeq, const char* readSeq, int errorThreshold,
              int* finalBitVector, int** hammingMasks, int* accepted) {
    int readLen = strlen(readSeq);
    int maxMasks = 2 * errorThreshold + 1;
    *accepted = 0;

    int* magnetMask = (int*)malloc(readLen * sizeof(int));
    for (int i = 0; i < readLen; i++) magnetMask[i] = 1;

    char *d_refSeq, *d_readSeq;
    int *d_allHammingMasks, *d_errorCounts;

    size_t seqSize = readLen + 1;
    cudaMalloc(&d_refSeq, seqSize * sizeof(char));
    cudaMalloc(&d_readSeq, seqSize * sizeof(char));
    cudaMalloc(&d_allHammingMasks, maxMasks * readLen * sizeof(int));
    cudaMalloc(&d_errorCounts, maxMasks * sizeof(int));

    cudaMemset(d_errorCounts, 0, maxMasks * sizeof(int));

    cudaMemcpy(d_refSeq, refSeq, seqSize * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_readSeq, readSeq, seqSize * sizeof(char), cudaMemcpyHostToDevice);

    dim3 blockSize(BLOCK_SIZE);
    dim3 gridSize((readLen + blockSize.x - 1) / blockSize.x, maxMasks);

    size_t sharedMemSize = (2 * readLen * sizeof(char)) + (BLOCK_SIZE * sizeof(int));

    computeHammingAndErrorsKernel<<<gridSize, blockSize, sharedMemSize>>>(
        d_refSeq, d_readSeq, d_allHammingMasks, d_errorCounts, readLen, errorThreshold);
    cudaGetLastError();

    int* h_errorCounts = (int*)malloc(maxMasks * sizeof(int));
    cudaMemcpy(h_errorCounts, d_errorCounts, maxMasks * sizeof(int), cudaMemcpyDeviceToHost);

    for (int i = 0; i < maxMasks; i++) {
        cudaMemcpy(hammingMasks[i], d_allHammingMasks + (i * readLen),
                          readLen * sizeof(int), cudaMemcpyDeviceToHost);

        if (h_errorCounts[i] <= errorThreshold) {
            *accepted = 1;
            memcpy(finalBitVector, magnetMask, readLen * sizeof(int));

            cudaFree(d_refSeq);
            cudaFree(d_readSeq);
            cudaFree(d_allHammingMasks);
            cudaFree(d_errorCounts);
            free(h_errorCounts);
            free(magnetMask);

            return 1;
        }
    }

    extractMagnetMask(hammingMasks, 0, readLen - 1, magnetMask, errorThreshold, readLen);
    memcpy(finalBitVector, magnetMask, readLen * sizeof(int));

    int errorCount = 0;
    for (int i = 0; i < readLen; i++) errorCount += magnetMask[i];

    *accepted = (errorCount <= errorThreshold) ? 1 : 0;

    cudaFree(d_refSeq);
    cudaFree(d_readSeq);
    cudaFree(d_allHammingMasks);
    cudaFree(d_errorCounts);
    free(h_errorCounts);
    free(magnetMask);

    return *accepted;
}

int readDataset(char sequences[MAX_SEQUENCES][MAX_SEQ_LEN],
               char refSequences[MAX_SEQUENCES][MAX_SEQ_LEN],
               int editThresholds[MAX_SEQUENCES], int* numSequences) {
    FILE* file = fopen("dataset.csv", "r");
    if (!file) {
        printf("Error: Cannot open dataset.csv\n");
        return 0;
    }

    char line[MAX_SEQ_LEN * 2 + 10];
    *numSequences = 0;

    while (fgets(line, sizeof(line), file) && *numSequences < MAX_SEQUENCES) {
        sscanf(line, "%[^,],%[^,],%d",
               sequences[*numSequences],
               refSequences[*numSequences],
               &editThresholds[*numSequences]);
        (*numSequences)++;
    }

    fclose(file);
    return 1;
}

void printPerformanceStats(int numSequences, double totalTime) {
    printf("Performance Statistics:\n");
    printf("Total Sequences Processed: %d\n", numSequences);
    printf("Total Processing Time: %.4f seconds\n", totalTime);
    printf("Average Time Per Sequence: %.4f seconds\n", totalTime / numSequences);
}

int main() {
    char sequences[MAX_SEQUENCES][MAX_SEQ_LEN];
    char refSequences[MAX_SEQUENCES][MAX_SEQ_LEN];
    int editThresholds[MAX_SEQUENCES];
    int numSequences;
    int accepted, result = 0;
    int check = 0;
    int errorCount;

    cudaDeviceReset();
    cudaFree(0);
    cudaSetDeviceFlags(cudaDeviceScheduleYield);

    if (!readDataset(sequences, refSequences, editThresholds, &numSequences)) {
        return 1;
    }

    int finalBitVector[MAX_SEQ_LEN] = {0};
    int* hammingMasks[2 * MAX_ERROR_THRESHOLD + 1];

    for (int i = 0; i < 2 * MAX_ERROR_THRESHOLD + 1; i++) {
        cudaMallocHost((void**)&hammingMasks[i], MAX_SEQ_LEN * sizeof(int));
    }

    double totalRunTime = 0.0;

    for (int run = 0; run < 10; run++) {
        double runTime = 0.0;
        int acceptedCount = 0, rejectedCount = 0;

        clock_t runStart = clock();

        for (int i = 0; i < numSequences; i++) {
            result = magnetCuda(refSequences[i], sequences[i], editThresholds[i],
                                finalBitVector, hammingMasks, &accepted);

            check = magnetCuda(refSequences[i], sequences[i], editThresholds[i],
                               finalBitVector, hammingMasks, &accepted);

            if (accepted) {
                acceptedCount++;
            } else {
                rejectedCount++;
            }

            if (result != check) {
                errorCount++;
            }
        }

        clock_t runEnd = clock();
        runTime = (double)(runEnd - runStart) / CLOCKS_PER_SEC;
        totalRunTime += runTime;

        printf("Run %d Time: %.6f seconds | Accepted: %d | Rejected: %d\n",
               run + 1, runTime, acceptedCount, rejectedCount);
    }

    double avgRunTime = totalRunTime / 10.0;  // Average time per full run

    printf("\n----- Statistics -----\n");
    printf("Total Runs: 10\n");
    printf("Total Execution Time: %.6f seconds\n", totalRunTime);
    printf("Average Execution Time per Run: %.6f seconds\n", avgRunTime);
    printf("Number of sequences processed: %d\n", numSequences);
    printf("Number of errors: %d\n", errorCount);

    for (int i = 0; i < 2 * MAX_ERROR_THRESHOLD + 1; i++) {
        cudaFreeHost(hammingMasks[i]);
    }

    cudaDeviceReset();
    return 0;
}

Overwriting magnetCUDA.cu


In [13]:
%%shell
nvcc -arch=sm_75 -o magnetCUDA magnetCUDA.cu
./magnetCUDA

Run 1 Time: 0.753937 seconds | Accepted: 346 | Rejected: 654
Run 2 Time: 0.526729 seconds | Accepted: 346 | Rejected: 654
Run 3 Time: 0.528331 seconds | Accepted: 346 | Rejected: 654
Run 4 Time: 0.516247 seconds | Accepted: 346 | Rejected: 654
Run 5 Time: 0.517962 seconds | Accepted: 346 | Rejected: 654
Run 6 Time: 0.512416 seconds | Accepted: 346 | Rejected: 654
Run 7 Time: 0.520007 seconds | Accepted: 346 | Rejected: 654
Run 8 Time: 0.522798 seconds | Accepted: 346 | Rejected: 654
Run 9 Time: 0.513808 seconds | Accepted: 346 | Rejected: 654
Run 10 Time: 0.515622 seconds | Accepted: 346 | Rejected: 654

----- Statistics -----
Total Runs: 10
Total Execution Time: 5.427857 seconds
Average Execution Time per Run: 0.542786 seconds
Number of sequences processed: 1000
Number of errors: 0
